In [1]:
import pandas as pd
import numpy as np
import shap

In [2]:
test_path = "../data/processed/test_inference.parquet"
raw_data_path = "../data/raw/hh_recsys_train_hh.pq"
audit_path = "dumps/audit/already_viewed__als_i2i__ials__top_selector__random_selector.parquet"

In [3]:
raw_data = pd.read_parquet(raw_data_path)
test_contest = pd.read_parquet(test_path)

In [4]:
raw_data = raw_data.rename(columns={
    "session_id": "target_session_id",
    "vacancy_id": "future_items",
    "action_type": "future_actions"
})[["user_id", "target_session_id", "future_items", "future_actions"]]

In [5]:
raw_data.head()

,user_id,target_session_id,future_items,future_actions
0,u_332060,s_28301374,"[v_2571684, v_488179, v_2389179, v_1393783, v_...","[2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, ..."
1,u_1057881,s_33868982,[v_665861],[2]
2,u_1036784,s_32474802,[v_2594840],[2]
3,u_786220,s_14060785,"[v_1473781, v_1622905, v_1621959, v_2289180, v...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, ..."
4,u_639152,s_23205986,"[v_695738, v_22433, v_1590524, v_502496, v_200...","[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, ..."


In [6]:
raw_data = raw_data.merge(test_contest[["user_id", "target_session_id"]], "inner", ["user_id", "target_session_id"])

In [7]:
raw_data = raw_data.explode(["future_items", "future_actions"])

In [8]:
raw_data = raw_data.rename(columns={"future_items": "target", "future_actions": "target_weight"})
raw_data.head()

,user_id,target_session_id,target,target_weight
0,u_216183,s_24850455,v_1616163,2
0,u_216183,s_24850455,v_1615001,2
0,u_216183,s_24850455,v_483847,2
0,u_216183,s_24850455,v_311932,2
0,u_216183,s_24850455,v_1805771,2


In [9]:
audit = pd.read_parquet(audit_path)

In [10]:
audit.shape # (19206353, 10)

(19028062, 9)

In [11]:
for col in audit.columns:
    print(col, audit[col].isna().sum())

user_id 0
session_id 0
vacancy_id 0
selector_id 0
is_viewed 0
is_bookmark 0
ials_cos 0
ials_dot 0
ials_item_norm 0


In [12]:
"""
    Теперь к рекомендациям приклеим реальные отклики
"""

audit_final = audit.merge(
    raw_data[["user_id", "target", "target_weight"]].rename(columns={"target": "vacancy_id"}),
    "inner",
    ["user_id", "vacancy_id"])

In [13]:
audit_final["target_weight"].value_counts()

target_weight
2    190028
1     59573
3      4249
Name: count, dtype: int64

In [89]:
# unknown_rows = audit.merge(
#     raw_data[["user_id", "target", "target_weight"]].rename(columns={"target": "vacancy_id"}),
#     "left",
#     ["user_id", "vacancy_id"])
# unknown_rows = unknown_rows[unknown_rows["target_weight"].isna()]
# unknown_rows = unknown_rows.sample(n=len(audit_final["selector_id"].unique()) * 50000, random_state=137)
# unknown_rows["target_weight"] = 0

In [90]:
# unknown_rows["target_weight"].value_counts()

In [91]:
# audit_final = pd.concat([audit_final, unknown_rows])

In [14]:
audit_final

,user_id,session_id,vacancy_id,selector_id,is_viewed,is_bookmark,ials_cos,ials_dot,ials_item_norm,target_weight
0,u_1000083,s_5984549,v_506297,0,1,0,0.000000,0.000000,0.000000,1
1,u_1000083,s_5984549,v_506297,0,1,0,0.000000,0.000000,0.000000,2
2,u_1000103,s_9861026,v_2614108,0,1,0,0.822553,0.944605,0.073486,2
3,u_1000103,s_9861026,v_2614108,2,1,0,0.822553,0.944605,0.073486,2
4,u_1000103,s_9861026,v_101375,2,0,0,0.796152,1.057088,0.084963,2
...,...,...,...,...,...,...,...,...,...,...
253845,u_999991,s_17039711,v_1387246,2,0,0,0.822706,0.708924,0.079743,2
253846,u_999991,s_17039711,v_1387246,1,0,0,0.822706,0.708924,0.079743,2
253847,u_999991,s_17039711,v_2098873,2,0,0,0.699876,0.291841,0.038589,2
253848,u_999991,s_17039711,v_2129070,2,0,0,0.764142,0.743925,0.090093,2


In [15]:
audit_final["target_weight"].value_counts()

target_weight
2    190028
1     59573
3      4249
Name: count, dtype: int64

In [16]:
"""
* action_type - тип события. 1 - отклик, 2 - просмотр (открытие страницы вакансии), 3 - добавление 
"""
action_type_2_target_weight = {
    1: 3,
    3: 1,
    2: 0
}

audit_final["target_weight"] = audit_final["target_weight"].map(action_type_2_target_weight)

In [17]:
audit_final.head()

,user_id,session_id,vacancy_id,selector_id,is_viewed,is_bookmark,ials_cos,ials_dot,ials_item_norm,target_weight
0,u_1000083,s_5984549,v_506297,0,1,0,0.000000,0.000000,0.000000,3
1,u_1000083,s_5984549,v_506297,0,1,0,0.000000,0.000000,0.000000,0
2,u_1000103,s_9861026,v_2614108,0,1,0,0.822553,0.944605,0.073486,0
3,u_1000103,s_9861026,v_2614108,2,1,0,0.822553,0.944605,0.073486,0
4,u_1000103,s_9861026,v_101375,2,0,0,0.796152,1.057088,0.084963,0


In [18]:
audit_final["selector_id"].value_counts()

selector_id
0    134256
2     62253
1     49249
4      8049
5        43
Name: count, dtype: int64

In [97]:
audit_final["target_weight"].value_counts()

target_weight
1    190028
5     59573
2      4249
Name: count, dtype: int64

In [20]:
audit_final_sampled = audit_final.groupby('selector_id').apply(lambda x: x.sample(n=min(25000, len(x)))).reset_index(drop = True)

In [21]:
audit_final_sampled["selector_id"].value_counts()

selector_id
0    25000
1    25000
2    25000
4     8049
5       43
Name: count, dtype: int64

In [22]:
audit_final_sampled["target_weight"].value_counts()

target_weight
0    60405
3    21212
1     1475
Name: count, dtype: int64

In [23]:
from catboost import CatBoostRanker, CatBoostRegressor
from sklearn.model_selection import train_test_split
from catboost import Pool
from random import shuffle

In [24]:
data = audit_final_sampled

user_ids = list(data["user_id"].unique())
shuffle(user_ids)

train_user_ids, test_user_ids = train_test_split(user_ids, test_size=0.15, shuffle=True, random_state=137)
train_user_ids, val_user_ids = train_test_split(train_user_ids, test_size=0.15, shuffle=True, random_state=137)

train_user_ids = pd.Series(train_user_ids).rename("user_id")
val_user_ids = pd.Series(val_user_ids).rename("user_id")
test_user_ids = pd.Series(test_user_ids).rename("user_id")

train_data = data.merge(train_user_ids, "inner", "user_id")
val_data = audit.merge(val_user_ids, "inner", "user_id")
test_data = audit.merge(test_user_ids, "inner", "user_id")

def get_pool(data):
    if "target_weight" in data.columns:
        dataset = Pool(data=data[['selector_id', 'is_viewed', 'is_bookmark', 'ials_cos', 'ials_dot', 'ials_item_norm']],
                         label=data["target_weight"])
    else:
        dataset = Pool(data=data[['selector_id', 'is_viewed', 'is_bookmark', 'ials_cos', 'ials_dot', 'ials_item_norm']])
    return dataset

In [37]:
def train_model(data, params):
    train_pool = get_pool(data)

    model = CatBoostRegressor(
        **params
    )

    model.fit(train_pool,
          verbose=False,
          plot=False)

    return model

def predict(data, model, labels):
    data = data.copy()
    predict_pool = get_pool(data)
    data["__predict"] = model.predict(predict_pool)

    data = data[["user_id", "session_id", "vacancy_id", "__predict"]]\
        .sort_values(by="__predict", ascending=False)

    data = data.groupby(["user_id", "session_id"])["vacancy_id"].apply(list).reset_index()
    data["vacancy_id"] = data["vacancy_id"].map(lambda x: x[:100])
    data = data.merge(labels, "inner", ["user_id", "session_id"])

    return data

def reciprocal_rank(recommendations, target):
    for idx, prediction in enumerate(recommendations):
        if prediction == target:
            return 1 / (idx + 1)

    return 0

def recall(recommendations, target):
    return int(target in set(recommendations))

def metrics(data):
    mrr = data[["vacancy_id", "target_vacancy_id"]].apply(
        lambda row: reciprocal_rank(row["vacancy_id"], row["target_vacancy_id"]),
        axis=1).mean()

    recall_mean = data[["vacancy_id", "target_vacancy_id"]].apply(
        lambda row: recall(row["vacancy_id"], row["target_vacancy_id"]),
        axis=1).mean()

    print({"mrr": mrr, "recall": recall_mean})

In [39]:
from itertools import product

dynamic_params = {
    "iterations": [350],
    "random_seed": [137],
    "learning_rate": [0.5, 0.1, 0.01, 0.001],
    "loss_function": ['RMSE', 'MAE', 'Poisson', 'Quantile'],
    "depth": [2, 4, 8],
}

param_names = list(dynamic_params.keys())
param_values = (zip(param_names, x) for x in product(*dynamic_params.values()))

labels = test_contest[["user_id", "session_id", "target_vacancy_id"]]

for paramset in param_values:
    kwargs = dict(paramset)
    model = train_model(train_data, kwargs)

    print(kwargs)
    metrics(predict(val_data, model, labels))

{'iterations': 350, 'random_seed': 137, 'learning_rate': 0.5, 'loss_function': 'RMSE', 'depth': 2}
{'mrr': 0.009123293597387304, 'recall': 0.16181878970244065}
{'iterations': 350, 'random_seed': 137, 'learning_rate': 0.5, 'loss_function': 'RMSE', 'depth': 4}
{'mrr': 0.011311868958208637, 'recall': 0.11400869274490137}
{'iterations': 350, 'random_seed': 137, 'learning_rate': 0.5, 'loss_function': 'RMSE', 'depth': 8}
{'mrr': 0.01559240553164452, 'recall': 0.1584754262788365}
{'iterations': 350, 'random_seed': 137, 'learning_rate': 0.5, 'loss_function': 'MAE', 'depth': 2}
{'mrr': 0.012196342339557489, 'recall': 0.15546639919759278}
{'iterations': 350, 'random_seed': 137, 'learning_rate': 0.5, 'loss_function': 'MAE', 'depth': 4}
{'mrr': 0.0, 'recall': 0.0}
{'iterations': 350, 'random_seed': 137, 'learning_rate': 0.5, 'loss_function': 'MAE', 'depth': 8}
{'mrr': 0.03498608052907356, 'recall': 0.17853560682046138}
{'iterations': 350, 'random_seed': 137, 'learning_rate': 0.5, 'loss_function': 

In [31]:
labels = test_contest[["user_id", "session_id", "target_vacancy_id"]]
metrics(predict(train_data, model, labels))
metrics(predict(val_data, model, labels))
metrics(predict(test_data, model, labels))

Mrr: 0.30045713828273696
Recall 0.38796105045736207
Mrr: 0.02345634008850733
Recall 0.15011701771982613
Mrr: 0.02718827540244617
Recall 0.161409491332765


In [ ]:
# Mrr: 0.3193923597272418
# Recall 0.4694175401353324
# Mrr: 0.027195831652761085
# Recall 0.14282134803307442
# Mrr: 0.028068189600038256
# Recall 0.14547390841320554

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer(test_data[['selector_id', 'is_viewed', 'is_bookmark', 'ials_cos', 'ials_dot', 'ials_item_norm']])

In [ ]:
shap.initjs()
shap.plots.force(shap_values[:1000], matplotlib=False)

In [32]:
final_model = train_model(data)

final_model.save_model(
    "dumps/production/catboost/model_v0"
)

/Users/platon.fedorov/opt/anaconda3/envs/hh_recsys/lib/python3.8/site-packages/catboost/core.py:6240: RuntimeWarning: Regression loss ('RMSE') ignores an important ranking parameter 'group_id'
  warnings.warn("Regression loss ('{}') ignores an important ranking parameter 'group_id'".format(loss_function), RuntimeWarning)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.8641005	total: 22.9ms	remaining: 6.84s
1:	learn: 0.8370454	total: 38.6ms	remaining: 5.75s
2:	learn: 0.8278713	total: 53.6ms	remaining: 5.31s
3:	learn: 0.8240042	total: 67.5ms	remaining: 5s
4:	learn: 0.8217906	total: 82.3ms	remaining: 4.86s
5:	learn: 0.8205194	total: 97ms	remaining: 4.75s
6:	learn: 0.8197422	total: 111ms	remaining: 4.65s
7:	learn: 0.8189511	total: 126ms	remaining: 4.59s
8:	learn: 0.8183879	total: 140ms	remaining: 4.52s
9:	learn: 0.8179279	total: 153ms	remaining: 4.42s
10:	learn: 0.8175829	total: 167ms	remaining: 4.39s
11:	learn: 0.8173216	total: 181ms	remaining: 4.34s
12:	learn: 0.8169727	total: 195ms	remaining: 4.3s
13:	learn: 0.8163264	total: 208ms	remaining: 4.24s
14:	learn: 0.8161027	total: 220ms	remaining: 4.18s
15:	learn: 0.8158231	total: 234ms	remaining: 4.16s
16:	learn: 0.8155523	total: 250ms	remaining: 4.16s
17:	learn: 0.8153152	total: 266ms	remaining: 4.17s
18:	learn: 0.8151046	total: 280ms	remaining: 4.15s
19:	learn: 0.8149647	total: 294ms	remaini